In [27]:
%load_ext autoreload
%autoreload 2

from sqlalchemy.orm import sessionmaker
from hansard.models import MP, Debate, SpokenContribution, Party, db_connect, create_table
import hansard.items
from sqlalchemy import exists

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from sqlalchemy.ext.declarative import declarative_base

In [4]:
party = hansard.items.Party(party='Labour')

In [5]:
mp = hansard.items.MP(
        name = "Dave",
        start_year = "1990",
        end_year = "present",
        constituency_last = "Derkenshire",
        house = "Upper",
        party = party
        )

In [6]:
labour = mp['party']

In [7]:
party_t = Party(party='Con')

In [8]:
party_t = Party(**labour)

In [10]:
party_t.party

'Labour'

In [11]:
Base = declarative_base()
Session = sessionmaker()
engine = db_connect()
create_table(engine)
Session.configure(bind=engine)
session = Session()

In [23]:
party = session.query(Party).filter_by(party=party_t.party).first()

In [28]:
mp_q = session.query(exists().where(MP.name=='Ms Diane Abbott')).scalar()

In [29]:
mp_q

False

In [49]:
q = session.query(Party).filter_by(party="Labour").first().party

In [50]:
q

'Labour'

In [71]:
q = session.query(MP).filter_by(name='Debbie Abrahams').first()

In [72]:
q

In [45]:
if q:
    print('woop')
else:
    print('nope')

nope


In [70]:
engine = db_connect()
create_table(engine)
Session.configure(bind=engine)
session = Session()

labour = Party(party='Con')
abbot = MP(name='Diane Abbot', start_year=1987, constituency_last='Hackney North and Stoke Newington',
           party=labour)
budget = Debate(debate_name='budget')
budget.mps = [abbot]
contrib = SpokenContribution(mp=abbot, text='Hear', debate=budget)

In [68]:
Party??

In [78]:
labour = Party(party='Labour')
abbot = MP(name='Diane Abbot', start_year=1987, constituency_last='Hackney North and Stoke Newington',
               party=labour)
budget = Debate(debate_name='budget')
budget.mps = [abbot]
contrib = SpokenContribution(mp=abbot, text="Hear", debate=budget)

InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers.  Original exception was: Could not determine join condition between parent/child tables on relationship Party.mps - there are no foreign keys linking these tables.  Ensure that referencing columns are associated with a ForeignKey or ForeignKeyConstraint, or specify a 'primaryjoin' expression.

In [62]:
session.add(labour)
session.commit()

UnmappedInstanceError: Class 'hansard.items.Party' is not mapped

In [36]:
class HansardPipeline(object):
    def __init__(self):
        engine = db_connect()
        create_table(engine)
        self.Session = sessionmaker(bind=engine)

    def process_item(self, item):
        session = self.Session()
        print("Attempting to add to DB")

        try:
            if type(item) is hansard.items.MP:
                mp = MP(**item)
                try:
                    session.add(mp)
                    session.commit()
                    print("Sucess!!")
                except:
                    session.rollback()
                    print("Failure...")
                finally:
                    session.close()
                    print("All done")
        except:
            raise
        return item

In [40]:
pipeline = HansardPipeline()

In [42]:
pipeline.process_item(mp)

Attempting to add to DB


InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers.  Original exception was: Could not determine join condition between parent/child tables on relationship Party.mps - there are no foreign keys linking these tables.  Ensure that referencing columns are associated with a ForeignKey or ForeignKeyConstraint, or specify a 'primaryjoin' expression.